참고

1. keybert와 kiwi 형태소분석기를 사용하여 키워드추출
- https://datainclude.me/posts/Keybert%EC%99%80_kiwi%ED%98%95%ED%83%9C%EC%86%8C%EB%B6%84%EC%84%9D%EA%B8%B0%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC_%ED%82%A4%EC%9B%8C%EB%93%9C%EC%B6%94%EC%B6%9C_%ED%95%98%EA%B8%B0/

2. kiwi document
- https://bab2min.github.io/kiwipiepy/v0.8.0/kr/
- https://mindscale.kr/course/text-mining/kiwi/


## Import Library

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import *
from tqdm import tqdm
from konlpy.tag import *

## Load Data

In [55]:
sample_raw = pd.read_csv('data/sample_description.csv', encoding="CP949")
sample = pd.read_csv('data/sample_description.csv', encoding="CP949")

sample["message"] = sample["message"].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]"," ",regex=True)
sample["description"] = sample["description"].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]"," ",regex=True)

In [56]:
sample

,user_id,message,description,reason,category
0,1,나만의 길을 찾을 때까지 인생이라는 게임의 레버를 당기는 법,좋아하는 일로 행복하게 일하는 상위 밀레니얼 프리워커 드로우앤드류입니다,자율성과 자유,경제적 독립
1,2,UI UX 아티클 백과사전 국내 해외 매일 업로드,년차 UX 컨설턴트이면서 년차 UX 강사로서 하루도 빠지지 않고 수집...,NaN,NaN
2,3,UXUI 취업 이직 스터디 포트폴리오 컨설팅,이런분들께 아주 도움되실꺼에요 UX디자이너가 되기위해 취업준비중인 취준생 ...,NaN,NaN
3,4,데일리 디자인 정보 공유 세션,매일 디자인 소식 하나를 픽해서 공유합니다 실무에 유용한 정보를 체계적으로 정리해...,NaN,NaN
4,5,피그마 학습방 서비스 기획자와 함께해요,피그마가 궁금하신가요 피그마를 쓰고 있으신가요 피그마를 사용하며 프로덕트를 만드...,NaN,NaN
5,6,UX Writer로 성장해볼까요,UX Writer 직무가 궁금하신 분들 현업에서는 어떻게 일하는지 궁금하신 분들 ...,NaN,NaN
6,7,BX 디자이너 취업 이직하기,신입 주니어 BX 브랜드 디자이너 분들의 구직을 위한 포트폴리오 코칭과 멘토링을...,NaN,NaN
7,8,성장하는 프로덕트 디자이너 SaaS 환영,SaaS와 프로덕트 디자인에 대한 정보와 의견을 나눠요 하루에 하나라도 제대로 읽...,NaN,NaN
8,9,디자이너로서 성공하고 싶은 사람들이 모인 방,안녕하세요 저는 고졸 국비지원 과정을 통해 스타트업 디자이너로 커리어를 시작하여...,NaN,NaN
9,10,프로토파이 Protopie 프로토타이핑 도움방,프로토타이핑 도구 중 high fidelity 높은 재현 을 가진 Protopie에...,NaN,NaN


## NLP 1-1.형태소 분석

In [108]:
from konlpy.tag import *

hannanum = Hannanum()
kkma = Kkma()
##komoran = Komoran()
okt = Okt()

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
stopwords = Stopwords()
kiwi = Kiwi()

In [116]:
kiwi.tokenize("분석 결과에서 불용어만 제외하고 출력할 수도 있다.", stopwords=stopwords)

[Token(form='분석', tag='NNG', start=0, len=2),
 Token(form='결과', tag='NNG', start=3, len=2),
 Token(form='불', tag='NNG', start=8, len=1),
 Token(form='용어', tag='NNG', start=9, len=2),
 Token(form='제외', tag='NNG', start=13, len=2),
 Token(form='출력', tag='NNG', start=18, len=2),
 Token(form='있', tag='VA', start=25, len=1)]

In [112]:
kiwi.analyze("분석 결과에서 불용어만 제외하고 출력할 수도 있다.")

[([Token(form='분석', tag='NNG', start=0, len=2),
   Token(form='결과', tag='NNG', start=3, len=2),
   Token(form='에서', tag='JKB', start=5, len=2),
   Token(form='불', tag='NNG', start=8, len=1),
   Token(form='용어', tag='NNG', start=9, len=2),
   Token(form='만', tag='JX', start=11, len=1),
   Token(form='제외', tag='NNG', start=13, len=2),
   Token(form='하', tag='XSV', start=15, len=1),
   Token(form='고', tag='EC', start=16, len=1),
   Token(form='출력', tag='NNG', start=18, len=2),
   Token(form='하', tag='XSV', start=20, len=1),
   Token(form='ᆯ', tag='ETM', start=20, len=1),
   Token(form='수', tag='NNB', start=22, len=1),
   Token(form='도', tag='JX', start=23, len=1),
   Token(form='있', tag='VA', start=25, len=1),
   Token(form='다', tag='EF', start=26, len=1),
   Token(form='.', tag='SF', start=27, len=1)],
  -81.8148422241211)]

In [ ]:
def noun_extractor_stopwords(text):
    results = []
    result = kiwi.tokeni(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

In [97]:
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

In [103]:
print(sample["description"][1])

print(hannanum.nouns(sample["description"][2]))
print(kkma.nouns(sample["description"][2]))
print(okt.nouns(sample["description"][2]))
print(noun_extractor(sample["description"][2]))

      년차 UX 컨설턴트이면서   년차 UX 강사로서 하루도 빠지지 않고 수집했던 방대한 양의 자료들을 공개하고자 합니다      매일 다양한 주제의 국내 해외 UI UX 아티클이 업로드됩니다      국내 해외 UI UX 트렌드를 빠르게 파악할 수 있습니다      구글 브런치 서핏 등에 검색하지 않고도 해시태그를 통해 아티클을 손쉽게 찾을 수 있습니다      지금 바로 입장하실 수 있습니다 
['이런분들께', '도움되실꺼에요', '디자이너', '되기위해', '취업준비중', '취준생', '전공자분', 'UX입문과정UX팀', '이직', '이직', '준비', '과정']
['분', '도움', '꺼', '디자이너', '취업', '취업준비중인', '준비', '중인', '취', '생', '타', '전공자', '전공자분', '분', '입문', '입문과정', '과정', '팀', '이직']
['분', '아주', '도움', '디자이너', '위해', '취업', '준비', '취준생', '타', '전공자', '위', '입문', '과정', '팀', '이직', '위', '이직', '준비', '과정']
['도움', 'UX', '디자이너', '취업', '준비', '취준생', '전공', 'UX', '입문', '과정', 'UX', '이직', '이직', '준비', '과정']


In [104]:
for i in range(0, len(sample)):
    print("\n ######################################## \n")
    print(sample["description"][i])
    print(hannanum.nouns(sample["description"][i]))
    print(kkma.nouns(sample["description"][i]))
    print(okt.nouns(sample["description"][i]))
    print(noun_extractor(sample["description"][i]))


 ######################################## 

좋아하는 일로 행복하게 일하는 상위    밀레니얼 프리워커 드로우앤드류입니다 
['일', '행복', '일', '상위', '밀레니얼', '프리워커', '드로우앤드류']
['일', '행복', '상위', '밀레', '얼', '프리', '프리워', '워', '드로우', '드로우앤드', '앤드']
['일로', '일', '상위', '밀레', '얼', '프리', '워커', '드', '로우', '앤드류']
['행복', '상위', '밀레니얼', '프리', '워커', '드로우', '앤드']

 ######################################## 

      년차 UX 컨설턴트이면서   년차 UX 강사로서 하루도 빠지지 않고 수집했던 방대한 양의 자료들을 공개하고자 합니다      매일 다양한 주제의 국내 해외 UI UX 아티클이 업로드됩니다      국내 해외 UI UX 트렌드를 빠르게 파악할 수 있습니다      구글 브런치 서핏 등에 검색하지 않고도 해시태그를 통해 아티클을 손쉽게 찾을 수 있습니다      지금 바로 입장하실 수 있습니다 
['년차', '컨설턴트', '년차', '강사', '하루', '수집', '방대한', '양', '자료들', '공개', '다양', '주제', '국내', '해외', '아티클', '업로드', '국내', '해외', '트렌드', '파악', '수', '구글', '브런치', '서핏', '등', '검색', '해시태그', '아티클', '수', '입장', '수']
['년', '년차', '차', '컨설턴트', '컨설턴트이면서', '이면서', '강사', '하루', '수집', '양의', '자료', '공개', '다양', '주제', '국내', '해외', '티', '업', '업로드', '로드', '국', '내', '트렌드', '파악', '수', '구', '구글', '글', '브런', '핏', '등에', '검색', '해시', '해시태그', '태그', '을', '입장']

['세상', '도움', '브랜드', '공부']
['세상', '도움', '브랜드', '만들기', '위해', '공부']
['세상', '도움', '브랜드', '공부']

 ######################################## 

스타트업에서 서비스와 제품을 성장시키는 창업가적 고민을 하는 분들의 모임입니다    고충을 서로 나누고 얕고 넓은 네트워킹을 위해 오프라인 커피 티타임을 희망하시는 분들 함께 모여요  
['스타트업', '서비스', '제품', '성장', '창업가적', '고민', '들', '모임', '고충', '네트워킹', '오프라', '커피', '티타임', '희망하시', '모']
['스타트', '스타트업', '업', '서비스', '제품', '성장', '창업', '창업가적', '가적', '고민', '분', '모임', '고충', '네트', '네트워킹', '워킹', '오프', '오프라인', '라인', '커피', '티타임', '희망']
['스타트업', '서비스', '제품', '성장', '창업가', '고민', '분', '모임', '고충', '서로', '네트워킹', '위해', '오프라인', '커피', '티타임', '희망']
['스타트업', '서비스', '제품', '성장', '창업가', '고민', '모임', '고충', '네트워킹', '오프라인', '커피', '티타임', '희망']

 ######################################## 

한국 홍콩에서 국내외 은행 증권사 자산운용사에서 프랍트레이딩   주식운용  파생상품  ELS 헤지운용 경험으로 금융기관에서 세일즈앤 트레이딩 Job을 찾으시는 분들에게 멘토링  
['한국', '홍콩', '국내외', '은행', '증권사', '자산운용사', '프랍트레이딩', '주식운용', '파생상품', '헤지운용', '경험', '금융기관', '세일즈앤', '트레이딩', '들', '멘토링']
['한국', '홍', '홍콩', '콩', '국내외', '은행', '증권사', '자산', '자산운용사', '운용'

['플랫폼', '불문', '애플', '생태계', '소프트', '소프트웨어', '웨어', '개발자', '환영']
['플랫폼', '불문', '애플', '생태계', '소프트웨어', '개발자', '모두', '환영']
['플랫폼', '불문', '애플', '생태', '소프트웨어', '개발자', '환영']

 ######################################## 

Vue js 배우거나 개발하고 있는 사람들 모임입니다  빠르게 배우고 어려움도 함께 해쳐나가 보시죠        홀릭스 채팅도 Vue  로 만들어졌어요  
['개발', '사람들', '모임', '어려움', '홀릭스', '채팅', '로']
['개발', '사람', '모임', '어려움', '나', '릭스', '채팅']
['개발', '사람', '모임', '어려움', '보시', '홀릭스', '채팅', '로']
['Vue', 'js', '개발', '사람', '모임', '어려움', '홀릭스', '채팅', 'Vue']

 ######################################## 

javascript 기반 Node js 노드 개발 대화  초보자부터 현직자까지 자유 참여 
['기반', '노드', '개발', '대화', '초보자', '현직자', '자유', '참여']
['기반', '노드', '개발', '대화', '초보자', '현직', '자유', '참여']
['기반', '노드', '개발', '대화', '초보자', '현직', '자유', '참여']
['javascript', '기반', 'Node', 'js', '노드', '개발', '대화', '초보자', '현직', '자유', '참여']

 ######################################## 

Android와 iOS 모바일 어플리케이션부터 웹과 Mac  linux 이젠 Windows까지  하나의 소스로 여러 플랫폼을 개발 할 수 있는 Dart와 Flutter 맛좀 보고
['모바', '어플리케이션', '웹', '이젠', '하나', '소

In [105]:
for i in range(0, len(sample)):
    print("\n ######################################## \n")
    print(sample["message"][i])
    print(hannanum.nouns(sample["message"][i]))
    print(kkma.nouns(sample["message"][i]))
    print(okt.nouns(sample["message"][i]))
    print(noun_extractor(sample["message"][i]))


 ######################################## 

나만의 길을 찾을 때까지 인생이라는 게임의 레버를 당기는 법
['나', '길', '때', '인생', '게임', '레버', '법']
['나', '길', '때', '인생', '게임', '레버', '법']
['나', '길', '때', '인생', '게임', '레버', '법']
['인생', '게임', '레버']

 ######################################## 

UI UX 아티클 백과사전  국내 해외    매일 업로드
['아티클', '백과사전', '국내', '해외', '업로드']
['티', '백과', '백과사전', '사전', '국내', '해외', '매일', '업', '업로드', '로드']
['아티', '백과사전', '국내', '해외', '매일', '업로드']
['UI', 'UX', '아티클', '백과사전', '국내', '해외', '업로드']

 ######################################## 

UXUI 취업 이직 스터디 포트폴리오 컨설팅
['취업', '이직', '스터디', '포트폴리오', '컨설팅']
['취업', '이직', '스터디', '포트폴리오', '컨설팅']
['취업', '이직', '스터디', '포트폴리오', '컨설팅']
['UXUI', '취업', '이직', '스터디', '포트폴리오', '컨설팅']

 ######################################## 

데일리 디자인 정보 공유 세션
['데', '리', '디자인', '정보', '공유', '세션']
['데일리', '디자인', '정보', '공유', '션']
['데', '일리', '디자인', '정보', '공유', '세션']
['데일리', '디자인', '정보', '공유', '세션']

 ######################################## 

피그마 학습방   서비스 기획자와 함께해요 
['피그', '학습방', '서비스', '기획자']
['피그

## NLP 1-2. 키워드 추출 + 형태소분석

In [84]:
from keybert import KeyBERT
kw_model = KeyBERT()

In [85]:
for i in range(0, len(sample)):
    print("\n ######################################## \n")
    print(sample["description"][i])
    print(kw_model.extract_keywords(sample["description"][i], top_n = 5))


 ######################################## 

좋아하는 일로 행복하게 일하는 상위    밀레니얼 프리워커 드로우앤드류입니다 
[('드로우앤드류입니다', 0.7664), ('밀레니얼', 0.6879), ('일하는', 0.5507), ('프리워커', 0.5347), ('일로', 0.4201)]

 ######################################## 

      년차 UX 컨설턴트이면서   년차 UX 강사로서 하루도 빠지지 않고 수집했던 방대한 양의 자료들을 공개하고자 합니다      매일 다양한 주제의 국내 해외 UI UX 아티클이 업로드됩니다      국내 해외 UI UX 트렌드를 빠르게 파악할 수 있습니다      구글 브런치 서핏 등에 검색하지 않고도 해시태그를 통해 아티클을 손쉽게 찾을 수 있습니다      지금 바로 입장하실 수 있습니다 
[('컨설턴트이면서', 0.4586), ('입장하실', 0.3733), ('공개하고자', 0.3445), ('검색하지', 0.3392), ('아티클을', 0.3027)]

 ######################################## 

  이런분들께 아주 도움되실꺼에요   UX디자이너가 되기위해 취업준비중인 취준생   타 전공자분들을 위한 UX입문과정UX팀으로 이직을 위한 이직 준비 과정
[('ux입문과정ux팀으로', 0.8483), ('전공자분들을', 0.5955), ('ux디자이너가', 0.5218), ('이직을', 0.4782), ('과정', 0.423)]

 ######################################## 

매일 디자인 소식 하나를 픽해서 공유합니다  실무에 유용한 정보를 체계적으로 정리해 드립니다  뼈가 되고 살이 되는 정보로 튼튼한 디자이너로 성장하세요 
[('공유합니다', 0.6521), ('디자이너로', 0.5608), ('드립니다', 0.5021), ('성장하세요', 0.4892), ('매일', 0.4598

[('모임입니다', 0.6359), ('주니어', 0.5954), ('막막하신', 0.5514), ('들어와서', 0.5449), ('나눠요', 0.5417)]

 ######################################## 

이미 Production에서 사용중인 개발자와 새로 공부하는 뉴비까지  안드로이드의 새로운 UI 프레임워크에 대한 최신 정보를 얻으세요 
[('production에서', 0.5564), ('사용중인', 0.5561), ('ui', 0.4718), ('프레임워크에', 0.4254), ('안드로이드의', 0.3962)]

 ######################################## 

플랫폼 불문 애플 생태계 소프트웨어 개발자들이라면 모두 환영합니다          
[('환영합니다', 0.7627), ('개발자들이라면', 0.6947), ('모두', 0.4928), ('애플', 0.4693), ('불문', 0.4527)]

 ######################################## 

Vue js 배우거나 개발하고 있는 사람들 모임입니다  빠르게 배우고 어려움도 함께 해쳐나가 보시죠        홀릭스 채팅도 Vue  로 만들어졌어요  
[('모임입니다', 0.5494), ('vue', 0.5268), ('배우거나', 0.4766), ('홀릭스', 0.4541), ('보시죠', 0.4484)]

 ######################################## 

javascript 기반 Node js 노드 개발 대화  초보자부터 현직자까지 자유 참여 
[('javascript', 0.6436), ('node', 0.5574), ('js', 0.5475), ('초보자부터', 0.3123), ('자유', 0.3067)]

 ######################################## 

Android와 iOS 모바일 어플리케이션부터 웹과 Mac  linux 이젠 Windows

In [106]:
for i in range(0, len(sample)):
    print("\n ######################################## \n")
    print(sample["description"][i])
    case1 = hannanum.nouns(sample["description"][i])
    case2 = kkma.nouns(sample["description"][i])
    case3 = okt.nouns(sample["description"][i])
    case4 = noun_extractor(sample["description"][i])
    
    case1 = ' '.join(case1)
    case2 = ' '.join(case2)
    case3 = ' '.join(case3)
    case4 = ' '.join(case4)
    
    print(case1)
    print(kw_model.extract_keywords(case1, keyphrase_ngram_range=(1, 1), top_n = 5))
    print(case2)
    print(kw_model.extract_keywords(case2, keyphrase_ngram_range=(1, 1), top_n = 5))
    print(case3)
    print(kw_model.extract_keywords(case3, keyphrase_ngram_range=(1, 1), top_n = 5))
    print(case4)
    print(kw_model.extract_keywords(case4, keyphrase_ngram_range=(1, 1), top_n = 5))


 ######################################## 

좋아하는 일로 행복하게 일하는 상위    밀레니얼 프리워커 드로우앤드류입니다 
일 행복 일 상위 밀레니얼 프리워커 드로우앤드류
[('드로우앤드류', 0.8837), ('밀레니얼', 0.7841), ('행복', 0.6281), ('프리워커', 0.5686), ('상위', 0.3857)]
일 행복 상위 밀레 얼 프리 프리워 워 드로우 드로우앤드 앤드
[('드로우앤드', 0.8555), ('프리워', 0.7233), ('드로우', 0.6323), ('밀레', 0.5306), ('앤드', 0.4699)]
일로 일 상위 밀레 얼 프리 워커 드 로우 앤드류
[('워커', 0.7405), ('일로', 0.6982), ('로우', 0.6513), ('앤드류', 0.5751), ('밀레', 0.5618)]
행복 상위 밀레니얼 프리 워커 드로우 앤드
[('밀레니얼', 0.7868), ('워커', 0.7243), ('드로우', 0.6457), ('앤드', 0.4887), ('행복', 0.4602)]

 ######################################## 

      년차 UX 컨설턴트이면서   년차 UX 강사로서 하루도 빠지지 않고 수집했던 방대한 양의 자료들을 공개하고자 합니다      매일 다양한 주제의 국내 해외 UI UX 아티클이 업로드됩니다      국내 해외 UI UX 트렌드를 빠르게 파악할 수 있습니다      구글 브런치 서핏 등에 검색하지 않고도 해시태그를 통해 아티클을 손쉽게 찾을 수 있습니다      지금 바로 입장하실 수 있습니다 
년차 컨설턴트 년차 강사 하루 수집 방대한 양 자료들 공개 다양 주제 국내 해외 아티클 업로드 국내 해외 트렌드 파악 수 구글 브런치 서핏 등 검색 해시태그 아티클 수 입장 수
[('컨설턴트', 0.4942), ('해외', 0.3857), ('검색', 0.3745), ('자료들', 0.3422), ('국내', 0.2916)]
년

[('업계', 0.5546), ('준비', 0.5165), ('디자인', 0.5047), ('씨앗', 0.49), ('경험', 0.4467)]
포트폴리오 디자인 지식 등 그래픽 디자인 대한 경험 디자인 업계 취 이직 준비 디자인 진심 나 그래픽 씨앗 어서
[('디자인', 0.5543), ('그래픽', 0.4101), ('경험', 0.3794), ('업계', 0.3244), ('지식', 0.2906)]
포트폴리오 디자인 지식 그래픽 디자인 경험 디자인 업계 이직 준비 디자인 진심 그래픽 씨앗
[('디자인', 0.5716), ('그래픽', 0.503), ('경험', 0.3227), ('업계', 0.3177), ('지식', 0.279)]

 ######################################## 

   B B SaaS 프로덕트를 만드는 메이커 분들 모두 환영합니다  경영진  기획자  디자이너  개발자  마케터  CX         커뮤니티에 참여하고     관련 아티클 및 자료     실무 팁     네트워킹의 기회     본인 프로덕트의 자유로운 홍보 기회 등을 얻어가세요 
프로덕트 메이커 환영 경영진 기획자 디자이너 개발자 마케터 커뮤니티 참여 관련 아티클 자료 실무 팁 네트워킹 기회 본인 프로덕트 자유로운 홍보 기회 등
[('개발자', 0.5645), ('디자이너', 0.5014), ('메이커', 0.4869), ('홍보', 0.4684), ('기획자', 0.4193)]
프로덕트 메이커 분 모두 환영 경 경영진 영진 기획자 디자이너 개발자 마 커뮤니티 참여 관련 티 자료 실무 팁 네트 네트워킹 워킹 기회 본인 자유 홍보 등
[('개발자', 0.559), ('홍보', 0.4889), ('메이커', 0.4802), ('경영진', 0.4223), ('디자이너', 0.4187)]
프로덕트 메이커 모두 환영 영진 기획 디자이너 개발자 마케터 커뮤니티 참여 관련 아티 및 자료 실무 팁 네트워킹 기회 본인 프로덕트 홍보 기회 등
[('디자이너', 0.5

[('디자이너', 0.5545), ('프로젝트', 0.5476), ('제작자', 0.5451), ('미디어아트', 0.5405), ('개발자', 0.5355)]
이야기 문화 문화예술분야 예술 분야 생성 생성인공지능 인공 인공지능 지능 발전 활용 아티스트 관점 보기 철학 과학 미술 새 이슈 나누기 사람 환영 가상현실 증강 증강현실 현실 관심 차 산업 이용 문화예술 비즈니스 프로젝트 기획 영감 사업 아이디어 도출 커뮤니티 안 결 이 팀 팀빌딩 빌딩 주기적 공유 트렌드 파악 다양 모집 방송 영상 종사자 미디어 미디어아트 아트 제작자 파인 파인아트 전공자 클래식 음악 공연 연출가 디자이너 전시 기획자 큐레이터 개발자 광고 홍보 마 공간 등
[('문화예술분야', 0.5788), ('문화예술', 0.5496), ('프로젝트', 0.5478), ('미디어아트', 0.54), ('생성인공지능', 0.5179)]
이야기 문화 예술 분야 생 인공 지능 발전 활용 아티스트 관점 생 성형 철학 과학 미술 대한 이슈 나누기 사람 환영 가상현실 증강현실 관심 차 산업 이용 문화 예술 비즈니스 프로젝트 기획 영감 사업 아이디어 도출 커뮤니티 안 결 이 팀 빌딩 기적 이슈 공유 트렌드 파악 분야 사람 모집 방송 영상 분야 종사 미디어아트 제작자 아트 전공자 클래식 음악 전공자 공연 기획 및 연출가 아티스트 디자이너 전시 기획 큐레이터 개발자 디자이너 광고 홍보 마케터 디자이너 공간 디자이너 등
[('디자이너', 0.5819), ('제작자', 0.5438), ('프로젝트', 0.5328), ('미디어아트', 0.5294), ('개발자', 0.524)]
이야기 문화 예술 분야 AI 생성 인공 지능 발전 활용 아티스트 관점 생성 AI 철학 과학 미술 이슈 사람 환영 가상현실 증강 현실 관심 산업 이용 문화 예술 비즈니스 프로젝트 기획 영감 사업 아이디어 도출 커뮤니티 빌딩 주기 이슈 공유 트렌드 파악 다양 분야 사람 모집 ex 방송 영상 분야 종사자 미디어아트 제작자 아트 전공자 클래식 음악 

사용 개발자 공부 뉴비까지 안드로이드 프레임워크 최신 정보 얻으
[('안드로이드', 0.5945), ('개발자', 0.4731), ('프레임워크', 0.337), ('정보', 0.3098), ('사용', 0.3068)]
사용 사용중인 중인 개발자 공부 뉴 뉴비 비 드로 드로이드 이드 프레임 프레임워크 워크 최신 정보
[('개발자', 0.598), ('프레임워크', 0.5814), ('최신', 0.4039), ('사용', 0.39), ('프레임', 0.3402)]
사용 개발자 공부 뉴비 안드로이드 프레임워크 대한 최신 정보
[('안드로이드', 0.6), ('개발자', 0.482), ('프레임워크', 0.3376), ('사용', 0.3141), ('공부', 0.2935)]
Production 사용 개발자 공부 뉴비 안드로이드 UI 프레임 워크 최신 정보
[('개발자', 0.4945), ('안드로이드', 0.4848), ('production', 0.4446), ('사용', 0.2875), ('정보', 0.2712)]

 ######################################## 

플랫폼 불문 애플 생태계 소프트웨어 개발자들이라면 모두 환영합니다          
플랫폼 불문 애플 생태계 소프트웨어 개발자들이 환영
[('애플', 0.5653), ('개발자들이', 0.5285), ('생태계', 0.4122), ('소프트웨어', 0.3811), ('플랫폼', 0.2844)]
플랫폼 불문 애플 생태계 소프트 소프트웨어 웨어 개발자 환영
[('애플', 0.604), ('개발자', 0.4699), ('소프트웨어', 0.4609), ('생태계', 0.3827), ('플랫폼', 0.3645)]
플랫폼 불문 애플 생태계 소프트웨어 개발자 모두 환영
[('애플', 0.5789), ('개발자', 0.4947), ('생태계', 0.4289), ('소프트웨어', 0.404), ('플랫폼', 0.3203)]
플랫폼 불문 애플 생태 소프트웨어 개발자 환영
[('애플', 0.

## NLP 2. word embedding (fasttext)